# Banded LD matrix

$$
\begin{align}
\Sigma & =
\left(
  \begin{array}{cccccccc}
    1 & \sigma & \sigma^2 & \dots & \sigma^K & 0 & \dots & 0 \\
    \sigma & 1 & \sigma & \dots & \sigma^{K-1} & \sigma^K & \dots & 0 \\
    \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & 0 & \dots & 0 & 0 & \dots & 1
  \end{array}
\right) \\
\Sigma_{i,j} & =
\begin{cases}
\sigma^{\left| i-j \right|} & \left| i-j \right| \le K \\
0 & \left| i-j \right| > K
\end{cases}
\end{align}
$$

In [3]:
import numpy as np
import numpy.random as nr

In [32]:
M = 100
N = 100000

K = 10
sigmas = [0.2, 0.3]

In [33]:
def generate_banded_sigma(sigma, M, K):
    Sigma = np.diag([1.]*M)
    s = 1
    for k in range(1,K+1):
        s *= sigma
        np.fill_diagonal(Sigma[k:,:], s)
        np.fill_diagonal(Sigma[:,k:], s)
    return(Sigma)

In [35]:
Banded = np.array([ generate_banded_sigma(s, M, K) for s in sigmas ])

# Causal effect sizes

In [24]:
rho_b = 0.8
Mc = 10

In [57]:
B = nr.multivariate_normal([0,0], [[1,rho_b],[rho_b,1]], Mc).T

In [42]:
Ic = nr.choice(M, Mc, False)

In [43]:
Mask_c = np.zeros(M, dtype=bool)
Mask_c[Ic] = True
Mask_n = ~ Mask_c

# True $\rho \left( g_1, g_2 \right)$

In [64]:
G = [np.linalg.inv(S[Mask_n][:,Mask_n]).dot(S[Mask_n][:,Mask_c].dot(b)) for b, S in zip(B, Sigma)]

In [101]:
np.corrcoef(G)

array([[ 1.        ,  0.91848509],
       [ 0.91848509,  1.        ]])

# Genotypes

In [103]:
X = [ nr.multivariate_normal([0]*M, S, N) for S in Sigma ]

In [104]:
Y = [x[:,Mask_c].dot(b) for b, x in zip(B, X)]

In [105]:
Ghat = [np.linalg.inv(x[:,Mask_n].T.dot(x[:,Mask_n])).dot(x[:,Mask_n].T.dot(y)) for x, y in zip(X, Y)]

In [106]:
np.corrcoef(Ghat)

array([[ 1.        ,  0.89806259],
       [ 0.89806259,  1.        ]])

In [7]:
def simple_lm(X, Y):
    XY = X.T.dot(Y)
    XX = X.T.dot(X)
    XX1 = np.linalg.inv(XX)
    return(XX1.dot(XY))

In [8]:
def simulate_rho_g_hat(Sigmas, rho_b=1, Mc=10, N=10000, ):
    M  = Sigmas[0].shape[0]

    bs = nr.multivariate_normal([0,0], [[1,rho_b],[rho_b,1]], Mc).T

    Ic  = nr.choice(M, Mc, False)

    Mask_c = np.zeros(M, dtype=bool)
    Mask_c[Ic] = True
    Mask_n = ~ Mask_c
    
    gs = [ np.linalg.inv(S[Mask_n][:,Mask_n]).dot(S[Mask_n][:,Mask_c].dot(b)) for b, S in zip(bs, Sigmas) ]
    rho_g = np.corrcoef(gs)[0,1]
    
    Xs = [ nr.multivariate_normal([0]*M, S, N) for S in Sigmas ]
    Ys = [ X[:,Mask_c].dot(b) for b, X in zip(bs, Xs) ]

    g_hats = [ simple_lm(X[:,Mask_n], Y) for X, Y in zip(Xs, Ys)]
    rho_g_hat = np.corrcoef(g_hats)[0,1]
    
    return(rho_g, rho_g_hat)

# Real LD matrix

In [1]:
files = [ '../phase2/ld/eas.eur.22.0.ld.bin', '../phase2/ld/eas.eur.22.1.ld.bin' ]

In [4]:
Sigmas = [ np.fromfile(f, 'double', -1, '') for f in files ]

In [5]:
M = int(np.sqrt(Sigmas[0].shape[0]))

In [6]:
rho_b = 0.8
Mc = 30

In [7]:
Sigmas = np.array([ S.reshape((M,M)) for S in Sigmas ])

In [8]:
l = 1
lI = np.diag(l*np.ones(M))
Sigmasx = Sigmas + np.array([lI, lI])

In [9]:
X = np.array([nr.multivariate_normal(np.zeros(M), S, 100) for S in Sigmasx])

In [10]:
Ic  = nr.choice(M, Mc, False)

In [11]:
Mask_c = np.zeros(M, dtype=bool)
Mask_c[Ic] = True
Mask_n = ~ Mask_c

In [12]:
G = np.corrcoef(X[0][:,Mask_n], X[1][:,Mask_n])

In [13]:
bs = nr.multivariate_normal([0,0], [[1,rho_b],[rho_b,1]], Mc).T

In [14]:
gs = [ np.linalg.inv(S[Mask_n][:,Mask_n]).dot(S[Mask_n][:,Mask_c].dot(b)) for b, S in zip(bs, Sigmasx) ]
rho_g = np.corrcoef(gs)[0,1]

In [15]:
rho_g

0.41632392984035699

In [25]:
trace1 = np.trace(Sigmasx[0,Mask_n,:].dot(Sigmasx[1,Mask_n,:].T))

In [27]:
trace1

13639.152069930289

In [95]:
Y = np.array([ x[:,Mask_c].dot(b) for b, x in zip(bs, X) ])

In [99]:
Y = Y.reshape((2,-1,1))

In [100]:
Y11 = Y[0].dot(Y[0].T)
Y12 = Y[0].dot(Y[1].T)
Y22 = Y[1].dot(Y[1].T)

In [107]:
G11 = G[:100][:,:100]
G12 = G[:100][:,100:]
G22 = G[100:][:,100:]

In [108]:
h1  = np.multiply( np.triu(G11,1), np.triu(Y11,1) ).sum() / np.triu(G11**2,1).sum()
h2  = np.multiply( np.triu(G22,1), np.triu(Y22,1) ).sum() / np.triu(G22**2,1).sum()
h12 = np.multiply( G12, Y12 ).sum() / G12.sum()

In [109]:
h12 / np.sqrt( h1 * h2 )

8.7272575327769175

# REML

In [ ]:
s